In [ ]:
%aiida

In [ ]:
import os
import numpy as np

from ipywidgets import interact

from aiida.orm import Code, DataFactory, CalculationFactory
from aiida.orm.data.base import Float, Str

from aiida.work.process_registry import ProcessRegistry
from aiida.work.run import run

SiestaCalculation = CalculationFactory('siesta.siesta')
PsfData = DataFactory('siesta.psf')
StructureData = DataFactory('structure')
ParameterData = DataFactory('parameter')
KpointsData = DataFactory('array.kpoints')

In [ ]:
code = None
def select_code(codename):
    global code
    code = Code.get_from_string(codename)

code_choice = interact(select_code, 
                       codename=Code.list_for_plugin('siesta.siesta'))

In [ ]:
filter_elements=("Mg", "O")

In [ ]:
pseudo_family = None
def select_pseudo_family(family):
    global pseudo_family
    pseudo_family = family
    
pseudo_family_choice = interact(select_pseudo_family,
                               family=PsfData.get_psf_groups(filter_elements=filter_elements))

In [ ]:
# MgO
alat = 4.117
cell = np.array([[0.0, 0.5, 0.5,],
                 [0.5, 0.0, 0.5,],
                 [0.5, 0.5, 0.0,],]) * alat
structure = StructureData(cell=cell)
structure.append_atom(
    position=(0.000 * alat, 0.000 * alat, 0.000 * alat),
    symbols=['Mg'])
structure.append_atom(
    position=(0.500 * alat, 0.500 * alat, 0.500 * alat),
    symbols=['O'])

In [ ]:
inputs = SiestaCalculation.process().get_inputs_template()
# Attach structure
inputs.structure = structure

In [ ]:
# Attach code
inputs.code = code
inputs._options.resources = {
    "num_machines": 1,
    "num_mpiprocs_per_machine": 1,
}
inputs._options.max_wallclock_seconds = 30 * 60

In [ ]:
kpoints_mesh = KpointsData()
kpoints_mesh.set_cell_from_structure(structure)
kpoints_mesh.set_kpoints_mesh_from_density(distance=0.15, offset=[0.5, 0.5, 0.5])

inputs.kpoints = kpoints_mesh

In [ ]:
bandskpoints = KpointsData()
bandskpoints.set_cell(structure.cell, structure.pbc)
bandskpoints.set_kpoints_path([
    ('K', 'G', 39),
    ('G', 'X', 37),
    ('X', 'W', 19),
    ('W', 'L', 27),
    ('L', 'G', 32),
])

inputs.bandskpoints = bandskpoints

In [ ]:
# Calculation parameters
parameters_dict = {
    'xc-functional': 'LDA',
    'xc-authors': 'CA',
    'spin-polarized': False,
    'meshcutoff': 200,  # Ry
#   'meshcutoff': '200 Ry',
    'max-scfiterations': 50,
}
inputs.parameters = ParameterData(dict=parameters_dict)

In [ ]:
# Basis set
#inputs.basis = ParameterData(dict={
#    'pao-energy-shift': '300 meV',
#    '%block pao-basis-sizes': 'O DZP',
#})

In [ ]:
# Pseudopotentials
from aiida_siesta.data.psf import get_pseudos_from_structure

inputs.pseudo = get_pseudos_from_structure(structure, pseudo_family.name)

In [ ]:
! verdi daemon restart

In [ ]:
JobCalc = SiestaCalculation.process()
result = run(JobCalc, **inputs)

In [ ]:
barray = result['bands_array']
barray.get_inputs()

In [ ]:
print barray.get_bands()